In [ ]:
%matplotlib inline

import numpy as np
import cv2
import matplotlib.pyplot as plt
import os

input_dir = "./INHS/"
output_dir = "./output_INHS/"

import os
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
    
padding = 0.05

for fileName in os.listdir(input_dir):
    print(os.path.join(input_dir, fileName))
    try:

        # read
        image = cv2.imread(os.path.join(input_dir, fileName))
        height, width, channels = image.shape
        original = image.copy()
        hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

        # gray scale
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

        # gamma (to make tail visible)
        post_gamma = np.array(255*(gray/255)**7,dtype='uint8')

        # blur
        blurred = cv2.GaussianBlur(post_gamma, (5, 5), 0)

        # edge detection
        canny = cv2.Canny(blurred, 50, 100, 5)
        kernel = np.ones((5,5),np.uint8)
        dilate = cv2.dilate(canny, kernel, iterations=5)

        # Find contours
        cnts = cv2.findContours(dilate, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        cnts = cnts[0] if len(cnts) == 2 else cnts[1]


        # find the image with highest hue.
        best = []
        bestDarkness = 0
        for c in cnts:
            x,y,w,h = cv2.boundingRect(c)
            if len(best) == 0 or (w > int(width/15) and h > int(height/15)):
                cropped_hsv_image = hsv_image[y:y+h, x:x+w]
                darkness = cv2.mean(cropped_hsv_image)[1];
                if darkness > bestDarkness:
                    bestDarkness = darkness
                    best = [x,y,w,h]


        # find region of interest
        x,y,w,h = best 
        x = int(x-padding*w)
        x = x if x >= 0 else 0
        y = int(y-padding*h)
        y = y if y >= 0 else 0
        w = int((1+ padding)*w)
        w = w if w > 0 and w < width else (1 if w <= 0 else width-1)
        h = int((1+padding)*h)
        h = h if h > 0 and h < height else (1 if h <= 0 else height-1)
        ROI = original[y:y+h, x:x+w]

        # Uncomment these two lines to see results in the notebook.
        plt.imshow(ROI)
        plt.show()
        
        # save the image
        cv2.imwrite(os.path.join(output_dir, fileName), ROI)
    except:
        print("image skipped")